In [1]:
import os

import numpy as np
from dipy.io.streamline import load_tractogram
from dipy.segment.clustering import QuickBundles
from dipy.io.pickles import save_pickle
from dipy.data import get_fnames

/home/teitxe/projects/tractoencoder_gsoc/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load FiberCup fibers (training, plausibles)

In [5]:
# Load tractogram
fname = "../data/fibercup/fibercup_advanced_filtering_no_ushapes/ae_input_std_endpoints/train/fibercup_Simulated_prob_tracking_minL10_resampled256_plausibles_std_endpoints_train.trk"
ref_anat = "../data/fibercup/Simulated_FiberCup_mask.nii.gz"
tractogram = load_tractogram(fname, ref_anat, bbox_valid_check=True)
streamlines = tractogram.streamlines

### Use QuickBundles to segment the bundles in FiberCup

In [6]:
qb = QuickBundles(threshold=10.)
clusters = qb.cluster(streamlines)
bundles = [np.array(bundle) for bundle in clusters]

print("Nb. clusters:", len(clusters))
print("Cluster sizes:", [len(cluster) for cluster in clusters])
print("Small clusters:", clusters < 10)

Nb. clusters: 7
Cluster sizes: [341, 592, 296, 256, 511, 448, 668]
Small clusters: [False False False False False False False]


### Save each bundle into a .trk file

In [7]:
import tractoencoder_gsoc.utils as utils
# Reference anatomy

output_path = "../data/fibercup/fibercup_advanced_filtering_no_ushapes/ae_input_std_endpoints/train/quickbundles"
os.makedirs(output_path, exist_ok=True)
for i, bundle in enumerate(bundles):
    bundle_name = f"bundle_{i}"
    bundle_trk_path = os.path.join(output_path, f"{bundle_name}.trk")
    utils.save_tractogram(streamlines=bundle,
                          tractogram_fname=bundle_trk_path,
                          img_fname=ref_anat)

2024-08-07 16:29:48.272105: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-07 16:29:48.469636: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-07 16:29:54.542807: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Load tractograms to check if saving was successfull

In [8]:
bundle_fname = "../data/fibercup/fibercup_advanced_filtering_no_ushapes/ae_input_std_endpoints/train/quickbundles/bundle_0.trk"
utils.prepare_tensor_from_file(bundle_trk_path, img_fname=ref_anat)

<tf.Tensor: shape=(668, 256, 3), dtype=float32, numpy=
array([[[ 4.6249304e+02, -1.5929135e+02,  7.9583731e+00],
        [ 4.6108334e+02, -1.6038249e+02,  8.0404186e+00],
        [ 4.5971027e+02, -1.6146280e+02,  8.3738937e+00],
        ...,
        [ 2.1730911e+02, -5.0642249e+02, -5.4129534e+00],
        [ 2.1679565e+02, -5.0810309e+02, -5.7078676e+00],
        [ 2.1689084e+02, -5.0984756e+02, -5.4661508e+00]],

       [[ 4.6913000e+02, -1.7114336e+02,  1.0161359e+00],
        [ 4.6828702e+02, -1.7262582e+02,  1.3278852e+00],
        [ 4.6754263e+02, -1.7397104e+02,  2.1477513e+00],
        ...,
        [ 2.3922229e+02, -5.0670642e+02, -5.0077481e+00],
        [ 2.3801962e+02, -5.0783148e+02, -5.4598289e+00],
        [ 2.3699394e+02, -5.0893256e+02, -6.3430505e+00]],

       [[ 4.4706940e+02, -1.4361279e+02,  2.6048899e-01],
        [ 4.4766782e+02, -1.4533060e+02,  4.2974496e-01],
        [ 4.4810315e+02, -1.4709525e+02,  5.8782077e-01],
        ...,
        [ 2.2274567e+02, -5.1009